## Experiment tracking
### wandb: https://wandb.ai/site



Primero, vamos a agregar experiment tracking utilizando wandb (Weights & Biases). Esto nos va a permitir monitorear los experimentos en tiempo real, guardar nuestros modelos , resultados, y podremos compartir experimentos con otros.



[Wandb collab full explained notebook ](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb#scrollTo=jufPgkgqz2eF)

## 👟 Run an experiment 

1.  **Start a new run** and pass in hyperparameters to track

2.  **Log metrics** from training or evaluation

3.  **Visualize results** in the dashboard

4. **Generate alerts** in the dashboard 

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import wandb
from wandb.keras import WandbCallback

wandb.login()

True

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import random

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# Launch 2 experiments, trying different dropout rates
for run in range(2):
    
    # Start a run, tracking hyperparameters
    wandb.init(
        project="ml-en-produccion",
        config={
        
            "activation_1": "relu",
            "dropout": random.uniform(0.01, 0.80),
            "optimizer": "adam",
            "loss": "categorical_crossentropy",
            "metric": "accuracy",
            "epoch": 3,
            "batch_size": 512,
        },
    )
    config = wandb.config
    
    # Cargar y normalizar el conjunto de datos 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Convertir las etiquetas en one-hot
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    # Define el preprocesamiento de la imagen

    data_augmentation = Sequential([
        preprocessing.Rescaling(1./255),
        preprocessing.RandomRotation(0.15),
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomZoom(0.2),
        preprocessing.RandomTranslation(0.1, 0.1),
        preprocessing.RandomContrast(0.2),
        preprocessing.RandomCrop(30, 30)
    ])

    # Define tu modelo
    model = tf.keras.models.Sequential([
        data_augmentation,  # Agrega las capas de preprocesamiento al inicio del modelo
        
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(config.dropout),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(config.dropout),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compila y entrena el modelo
    model.compile(optimizer=config.optimizer, loss=config.loss,  metrics=[config.metric])

    # Add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints

    wandb_callbacks = [
        WandbMetricsLogger(),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
    ]

    history = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epoch, validation_data=(x_test, y_test), callbacks=wandb_callbacks)
    
    wandb.finish()

2023-06-04 22:32:57.110309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/3
98/98 [==============================] - ETA: 0s - loss: 2.1798 - accuracy: 0.2853

2023-06-04 22:35:03.398736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'rescaling_input' with dtype uint8 and shape [?,32,32,3]
	 [[{{node rescaling_input}}]]
2023-06-04 22:35:03.438760: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,3]
	 [[{{node inputs}}]]
2023-06-04 22:35:03.450622: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,3]
	 [[{{no

INFO:tensorflow:Assets written to: my_model_01/assets


INFO:tensorflow:Assets written to: my_model_01/assets
wandb: Adding directory to artifact (./my_model_01)... Done. 0.1s


...

2023-06-04 22:47:08.663798: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'rescaling_1_input' with dtype uint8 and shape [?,32,32,3]
	 [[{{node rescaling_1_input}}]]
2023-06-04 22:47:08.702498: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,3]
	 [[{{node inputs}}]]
2023-06-04 22:47:08.717988: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,3]
	 [[

INFO:tensorflow:Assets written to: my_model_03/assets


INFO:tensorflow:Assets written to: my_model_03/assets
wandb: Adding directory to artifact (./my_model_03)... Done. 0.2s


98/98 [==============================] - 129s 1s/step - loss: 1.5910 - accuracy: 0.4288 - val_loss: 2.8999 - val_accuracy: 0.1409


epoch/accuracy,▁▆█
epoch/epoch,▁▅█
epoch/learning_rate,▁▁▁
epoch/loss,█▃▁
epoch/val_accuracy,▁█▆
epoch/val_loss,▄█▁
epoch/accuracy,0.4288
epoch/epoch,2
epoch/learning_rate,0.001
epoch/loss,1.59097
epoch/val_accuracy,0.1409


In [5]:
#save model 
model.save('model.h5')

#  W&B Alerts

**[W&B Alerts](https://docs.wandb.ai/guides/track/alert)** allows you to send alerts, triggered from your Python code, to your Slack or email. There are 2 steps to follow the first time you'd like to send a Slack or email alert, triggered from your code:

1) Turn on Alerts in your W&B [User Settings](https://wandb.ai/settings)

2) Add `wandb.alert()` to your code:

```python
wandb.alert(
    title="Low accuracy", 
    text=f"Accuracy is below the acceptable threshold"
)
```

In [4]:
import random 

# Start a wandb run
wandb.init(project="alerts-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f"Accuracy is: {accuracy}, {acc_threshold}")

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title="Low Accuracy",
            text=f"Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}",
        )
        print("Alert triggered")
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 1.912, 0.3
Accuracy is: 1.281, 0.3
Accuracy is: 1.139, 0.3
Accuracy is: 0.775, 0.3
Accuracy is: 1.87, 0.3
Accuracy is: 1.201, 0.3
Accuracy is: 0.646, 0.3
Accuracy is: 1.095, 0.3
Accuracy is: 1.197, 0.3
Accuracy is: 0.662, 0.3
Accuracy is: 0.434, 0.3
Accuracy is: 1.221, 0.3
Accuracy is: 0.647, 0.3
Accuracy is: 0.627, 0.3
Accuracy is: 0.394, 0.3
Accuracy is: 1.199, 0.3
Accuracy is: 0.993, 0.3
Accuracy is: 0.916, 0.3
Accuracy is: 1.047, 0.3
Accuracy is: 1.687, 0.3
Accuracy is: 0.861, 0.3
Accuracy is: 1.165, 0.3
Accuracy is: 0.298, 0.3
Alert triggered


Accuracy,█▅▅▃█▅▃▄▅▃▂▅▃▂▁▅▄▄▄▇▃▅▁
Accuracy,0.298


# H Tuning - wandb

In [ ]:
sweep_config = {
    'method': 'random',  # el método de búsqueda de hiperparámetros
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'learning_rate': {
            'values': [0.1, 0.01, 0.001, 0.0001]
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, project="Htuning")

def train():
    run = wandb.init()
    config = run.config

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=10, validation_data=(x_test, y_test), callbacks=[WandbCallback()])

wandb.agent(sweep_id, function=train)

In [6]:
#load model 
from tensorflow.keras.models import load_model
model = load_model('model.h5')

#  Gradio

https://gradio.app/

Gradio is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!


In [ ]:
#!pip install gradio

In [7]:
def classify_image(image):
    
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)

    # Obtiene las etiquetas de las clases
    class_labels = ['plane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    # Devuelve un diccionario con las etiquetas de las clases y sus probabilidades correspondientes
    return {class_labels[i]: float(prediction[0][i]) for i in range(10)}

In [8]:
import gradio as gr

iface = gr.Interface(
    fn=classify_image,  # la función que hace la clasificación
    inputs=gr.inputs.Image(shape=(32, 32)),  # el tipo de entrada que espera tu modelo
    outputs=gr.outputs.Label(num_top_classes=3),  # el tipo de salida que produce tu modelo
)
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://33a6c279773a0e32fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 39ms/step


In [ ]:
#pip uninstall markupsafe

# Quantization

La cuantificación se refiere a reducir la precisión numérica de los pesos y las activaciones de un modelo, lo que permite reducir el tamaño y mejorar la eficiencia del modelo. 

TensorFlow proporciona herramientas y APIs para realizar la cuantificación de manera sencilla.

In [10]:
!pip install tensorflow_model_optimization

     |████████████████████████████████| 241 kB 6.9 MB/s eta 0:00:01


In [11]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

2023-06-04 22:53:21.282585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'rescaling_1_input' with dtype uint8 and shape [?,32,32,3]
	 [[{{node rescaling_1_input}}]]
2023-06-04 22:53:21.318405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,3]
	 [[{{node inputs}}]]
2023-06-04 22:53:21.329403: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,3]
	 [[

INFO:tensorflow:Assets written to: /tmp/tmpmoyvznn6/assets


INFO:tensorflow:Assets written to: /tmp/tmpmoyvznn6/assets
2023-06-04 22:53:28.557030: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-04 22:53:28.557076: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-04 22:53:28.557502: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpmoyvznn6
2023-06-04 22:53:28.567166: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-04 22:53:28.567202: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpmoyvznn6
2023-06-04 22:53:28.597832: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-04 22:53:28.606356: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-04 22:53:28.752549: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

#### Probamos: quantization

In [12]:
def classify_image_2(image):
    
    image = np.expand_dims(image, axis=0)

    interpreter = tf.lite.Interpreter(model_path="quantized_model.tflite")
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], image)

    # Ejecuta la inferencia
    interpreter.invoke()

    # Obtiene el resultado de la inferencia
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Obtiene las etiquetas de las clases
    class_labels = ['plane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    # Devuelve un diccionario con las etiquetas de las clases y sus probabilidades correspondientes
    return {class_labels[i]: float(prediction[0][i]) for i in range(10)}

In [14]:
import gradio as gr

iface = gr.Interface(
    fn=classify_image_2,  # la función que hace la clasificación
    inputs=gr.inputs.Image(shape=(32, 32)),  # el tipo de entrada que espera tu modelo
    outputs=gr.outputs.Label(num_top_classes=3),  # el tipo de salida que produce tu modelo
)
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://83db79c4f5b8939e5c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Pruning

El pruning, o poda, es una técnica utilizada para reducir el tamaño de un modelo eliminando los pesos que son pequeños o cero.

El concepto es similar a la poda en la jardinería, donde se eliminan las ramas innecesarias para mantener el árbol saludable.

Esto puede mejorar la eficiencia del modelo en términos de velocidad y tamaño de almacenamiento, a veces con un coste mínimo en términos de precisión.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

# Cargar y normalizar el conjunto de datos 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define el preprocesamiento de la imagen

data_augmentation = Sequential([
    preprocessing.Rescaling(1./255),
    preprocessing.RandomRotation(0.15),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomTranslation(0.1, 0.1),
    preprocessing.RandomContrast(0.2),
    preprocessing.RandomCrop(30, 30)
])

# Define tu modelo
model = tf.keras.models.Sequential([
    data_augmentation,  # Agrega las capas de preprocesamiento al inicio del modelo
    
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compila y entrena el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_test, y_test))

# Definir el esquema de pruning. En este caso, comenzamos sin pruning y terminamos con el 50% de los pesos pruned. 
# El pruning comienza en el paso 2000 y termina en el paso 4000.
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                   final_sparsity=0.5,
                                                   begin_step=2000,
                                                   end_step=4000)
}

# Envolver el modelo con las capas de pruning. Esto agregará una operación de pruning a cada capa de nuestro modelo.
model_for_pruning = sparsity.prune_low_magnitude(model, **pruning_params)

# Necesitamos recompilar el modelo después de agregar las capas de pruning.
model_for_pruning.compile(optimizer='adam', 
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

# Entrenar el modelo con callbacks para habilitar pruning. 
# El callback UpdatePruningStep asegurará que el estado del pruning se actualiza en cada paso de entrenamiento.
callbacks = [
  sparsity.UpdatePruningStep(),
]

# Ajustar el modelo con nuestros datos de entrenamiento. Durante este entrenamiento, los pesos del modelo serán pruned según el esquema que definimos antes.
model_for_pruning.fit(x_train, y_train,
                      epochs=2,
                      callbacks=callbacks)

# Guarda el modelo#
model_for_pruning.save('pruning_conv.h5')